![email inbox](email_inbox.jpg)

Every day, professionals wade through hundreds of emails, from urgent client requests to promotional offers. It's like trying to find important messages in a digital ocean. But AI can help you stay afloat by automatically sorting emails to highlight what matters most.

You've been asked to build an intelligent email assistant using Llama, to help users automatically classify their incoming emails. Your system will identify which emails need immediate attention, which are regular updates, and which are promotions that can wait or be archived.

### The Data
You'll work with a dataset of various email examples, ranging from urgent business communications to promotional offers. Here's a peek at what you'll be working with:

### email_categories_data.csv

 Column | Description |
|--------|-------------|
| email_id | A unique identifier for each email in the dataset. |
| email_content | The full email text including subject line and body. Each email follows a format of "Subject" followed by the message content on a new line. |
| expected_category | The correct classification of the email: `Priority`, `Updates`, or `Promotions`. This will be used to validate your model's performance. |



In [93]:
# Run the following cells first
# Install necessary packages
!pip install llama-cpp-python==0.2.82 -q -q -q

In [94]:
# Download the model
!wget -q https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v0.3-GGUF/resolve/main/tinyllama-1.1b-chat-v0.3.Q4_K_M.gguf?download=true -O model.gguf

In [95]:
# Import required libraries
import pandas as pd
from llama_cpp import Llama

In [96]:
# Load the email dataset
emails_df = pd.read_csv('data/email_categories_data.csv')
# Display the first few rows of our dataset
print("Preview of our email dataset:")
emails_df.head(2)

Preview of our email dataset:


,email_id,email_content,expected_category
0,1,Urgent: Server Maintenance Required\nOur main ...,Priority
1,2,50% Off Spring Collection!\nDon't miss our big...,Promotions


In [97]:
#initializing llm
llm = Llama(model_path="model.gguf")

llama_model_loader: loaded meta data with 20 key-value pairs and 201 tensors from model.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = py007_tinyllama-1.1b-chat-v0.3
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 64
llama_model_loader: - kv   7:                 llama.attention.head_count u32             

In [98]:
#System prompt with few shot prompting

prompt = """Classify the emails into Priority,Promotions and Updates:
Example 1:
Urgent:Password Reset Required
Your account security requires immediate attention. Please reset your password within 24 hours.Priority

Example 2:
Special Offer - 50% Off Everything!
Don't miss our biggest sale of the year. Everything must go!Promotions

Example 3:
Canceled Event - Team Meeting
This event has been canceled and removed from your calendar.Updates

Example 4:
"""

In [99]:
def process_message(llm, message, prompt):
    """Process a message and return the response"""
    input_prompt = f"{prompt}\n{message}"
    response = llm(
        input_prompt,
        max_tokens=5,
        temperature=0.2,
        stop=["Q:","\n"],
    )
    return response['choices'][0]['text'].strip()

In [100]:
#testing emails from the dataset top 2 of different categories
test_emails=emails_df.head(2)

#processing each test mails and storing their results
results=[]
#iterating through the dataframe rows
for idx,row in test_emails.iterrows():
    email_content=row['email_content']
    expected_category=row['expected_category']
    #Getting model's classification
    result = process_message(llm, email_content, prompt)
    #store results
    results.append({
        'email_content': email_content,
        'expected_category':expected_category,
        'model_outcome':result
    })


llama_print_timings:        load time =    7524.99 ms
llama_print_timings:      sample time =       0.52 ms /     3 runs   (    0.17 ms per token,  5813.95 tokens per second)
llama_print_timings: prompt eval time =    7524.78 ms /   156 tokens (   48.24 ms per token,    20.73 tokens per second)
llama_print_timings:        eval time =     209.22 ms /     2 runs   (  104.61 ms per token,     9.56 tokens per second)
llama_print_timings:       total time =    7736.55 ms /   158 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    7524.99 ms
llama_print_timings:      sample time =       0.69 ms /     4 runs   (    0.17 ms per token,  5763.69 tokens per second)
llama_print_timings: prompt eval time =    2390.12 ms /    29 tokens (   82.42 ms per token,    12.13 tokens per second)
llama_print_timings:        eval time =     309.81 ms /     3 runs   (  103.27 ms per token,     9.68 tokens per second)
llama_print_timings:       total time =    2790.17 ms /    32 

In [101]:
#making dataframe of results
results_df=pd.DataFrame(results)

In [102]:
results_df

,email_content,expected_category,model_outcome
0,Urgent: Server Maintenance Required\nOur main ...,Priority,Priority
1,50% Off Spring Collection!\nDon't miss our big...,Promotions,Promotions


In [103]:
result1 = "Priority"
result2 = "Promotions"
# Displaying results
print(f"\nClassification Results: \n email 1  {result1} \n email 2: {result2}")


Classification Results: 
 email 1  Priority 
 email 2: Promotions
